In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install ultralytics



Cloning into 'yolov5'...
remote: Enumerating objects: 17493, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17493 (delta 1), reused 0 (delta 0), pack-reused 17490 (from 3)
Receiving objects: 100% (17493/17493), 16.59 MiB | 16.82 MiB/s, done.
Resolving deltas: 100% (11992/11992), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="0si8fSHWSgCajdynx6xe")
project = rf.workspace("oss-jbq3v").project("food_1-nlzrz")
version = project.version(4)
dataset = version.download("yolov5")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to food_1-4 in yolov5pytorch:: 100%|██████████| 2340/2340 [00:00<00:00, 4773.65it/s]


In [5]:
import os
import shutil
import random

# 정확한 경로 지정
base_path = "/content/yolov5/food_1-4/train"
image_dir = os.path.join(base_path, "images")
label_dir = os.path.join(base_path, "labels")

# 비율 설정
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# 파일 준비
image_files = [f for f in os.listdir(image_dir) if f.endswith((".jpg", ".png"))]
random.shuffle(image_files)

total = len(image_files)
n_train = int(total * train_ratio)
n_val = int(total * val_ratio)

train_files = image_files[:n_train]
val_files = image_files[n_train:n_train + n_val]
test_files = image_files[n_train + n_val:]

# 디렉토리 생성
for split in ["train", "val", "test"]:
    os.makedirs(f"/content/yolodata/images/{split}", exist_ok=True)
    os.makedirs(f"/content/yolodata/labels/{split}", exist_ok=True)

# 파일 이동 함수
def move_files(file_list, split):
    for file in file_list:
        image_src = os.path.join(image_dir, file)
        label_name = file.rsplit(".", 1)[0] + ".txt"
        label_src = os.path.join(label_dir, label_name)

        image_dst = f"/content/yolodata/images/{split}/{file}"
        label_dst = f"/content/yolodata/labels/{split}/{label_name}"

        if os.path.exists(image_src) and os.path.exists(label_src):
            shutil.copy(image_src, image_dst)
            shutil.copy(label_src, label_dst)

# 실행
move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

print("✅ 8:1:1 데이터 분할 완료")


✅ 8:1:1 데이터 분할 완료


In [7]:
with open("/content/yolov5/food_1-4/data.yaml", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        if line.strip().startswith("names:"):
            print(line)
        elif line.strip().startswith("-") or line.strip().startswith("'"):
            print(line)


names:

- '1'

- albap

- bossam

- gyeongdan

- jajangmyeon

- jjamppong

- kalguksu

- kimbab

- kimchifr

- mandu

- mulnaengmyeon

- pizza

- sikhye

- yubuchobap



In [9]:
yaml_content = """
train: ./images/train
val: ./images/val
test: ./images/test

nc: 13
names: [albap, bossam, gyeongdan, jajangmyeon, jjamppong, kalguksu, kimbab, kimchifr, mandu, mulnaengmyeon, pizza, sikhye, yubuchobap]
"""

with open("/content/yolov5/data.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_content.strip())

print("✅ 리스트 형식으로 수정된 data.yaml 저장 완료!")


✅ 리스트 형식으로 수정된 data.yaml 저장 완료!


In [13]:
!python /content/yolov5/train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/yolov5/food_1-4/data.yaml \
  --weights yolov5m.pt \
  --hyp data/hyps/hyp.scratch-med.yaml \
  --project runs/train \
  --name food_detect


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
      63/99      7.55G    0.03866    0.02231   0.005862        104        640:  70% 46/66 [00:22<00:10,  1.90it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      63/99      7.55G    0.03874    0.02265   0.005787        188        640:  71% 47/66 [00:23<00:12,  1.50it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      63/99      7.55G    0.03882    0.02289   0.005839        149        640:  73% 48/66 [00:24<00:11,  1.63it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      63/99      7.55G    0.03884    0.02299    0.00

In [12]:
import os, random, shutil

# 원본 이미지/라벨 폴더
base = "/content/yolov5/food_1-4/train"
img_dir = os.path.join(base, "images")
lbl_dir = os.path.join(base, "labels")

# 새 validation 경로 생성
val_img_dir = "/content/yolov5/food_1-4/valid/images"
val_lbl_dir = "/content/yolov5/food_1-4/valid/labels"
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

# 10% validation으로 이동
images = os.listdir(img_dir)
random.shuffle(images)
val_count = int(len(images) * 0.1)
val_images = images[:val_count]

for img in val_images:
    lbl = img.replace('.jpg', '.txt').replace('.png', '.txt')
    shutil.move(os.path.join(img_dir, img), os.path.join(val_img_dir, img))
    shutil.move(os.path.join(lbl_dir, lbl), os.path.join(val_lbl_dir, lbl))

print(f"✅ {val_count}개의 validation 이미지가 이동되었습니다.")


✅ 116개의 validation 이미지가 이동되었습니다.


In [14]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
import os
import shutil

# 실험 경로
base_path = "/content/yolov5/runs/train"
drive_path = "/content/drive/MyDrive/best.pt"

# 폴더 중 가장 마지막으로 수정된 경로 찾기
exp_paths = [os.path.join(base_path, d) for d in os.listdir(base_path)]
exp_paths = [p for p in exp_paths if os.path.isdir(p)]
latest_exp = max(exp_paths, key=os.path.getmtime)

# best.pt 경로
best_weight_path = os.path.join(latest_exp, "weights", "best.pt")

# 복사
if os.path.exists(best_weight_path):
    shutil.copy(best_weight_path, drive_path)
    print(f"✅ best.pt 저장 완료: {drive_path}")
else:
    print("❌ best.pt 파일을 찾을 수 없습니다.")


✅ best.pt 저장 완료: /content/drive/MyDrive/best.pt


In [19]:
from google.colab import files
files.download('/content/food_detect.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
!zip -r /content/food_detect.zip /content/yolov5/runs/train/food_detect


  adding: content/yolov5/runs/train/food_detect/ (stored 0%)
  adding: content/yolov5/runs/train/food_detect/weights/ (stored 0%)
  adding: content/yolov5/runs/train/food_detect/events.out.tfevents.1750346234.fa82fa99cad3.4999.0 (deflated 9%)
  adding: content/yolov5/runs/train/food_detect/opt.yaml (deflated 50%)
  adding: content/yolov5/runs/train/food_detect/hyp.yaml (deflated 43%)
